<a href="https://colab.research.google.com/github/inverter404/bosch-interiit/blob/main/BlackBox_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BlackBox Setting Evaluation loop notebook

## Importing libraries

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np
import os
import pandas as pd
import warnings
warnings.filterwarnigns('ignore')
from skimage.transform import resize

import torch
from torch import nn
from torch.nn import Parameter
from torch.nn import functional as F
from torch.nn import init

import torch
import json
from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
    UniformCropVideo
)
from typing import Dict

!pip -q install pytorch_ranger

from tqdm.notebook import tqdm

from pytorch_ranger import Ranger
from sklearn.metrics import classification_report

In [ ]:
!pip freeze | grep tensorflow

tensorflow @ file:///tensorflow-2.8.0-cp37-cp37m-linux_x86_64.whl
tensorflow-datasets==4.0.1
tensorflow-estimator==2.8.0
tensorflow-gcs-config==2.8.0
tensorflow-hub==0.12.0
tensorflow-io-gcs-filesystem==0.24.0
tensorflow-metadata==1.7.0
tensorflow-probability==0.16.0


In [ ]:
!pip freeze | grep torch

torch @ https://download.pytorch.org/whl/cu111/torch-1.10.0%2Bcu111-cp37-cp37m-linux_x86_64.whl
torchaudio @ https://download.pytorch.org/whl/cu111/torchaudio-0.10.0%2Bcu111-cp37-cp37m-linux_x86_64.whl
torchsummary==1.5.1
torchtext==0.11.0
torchvision @ https://download.pytorch.org/whl/cu111/torchvision-0.11.1%2Bcu111-cp37-cp37m-linux_x86_64.whl


In [ ]:
!pip freeze | grep sklearn

sklearn==0.0
sklearn-pandas==1.8.0


In [ ]:
!pip freeze | grep scikit-learn

scikit-learn==1.0.2


In [ ]:
!pip freeze | grep pandas

pandas==1.3.5
pandas-datareader==0.9.0
pandas-gbq==0.13.3
pandas-profiling==1.4.1
sklearn-pandas==1.8.0


In [ ]:
!pip freeze | grep cv

cvxopt==1.2.7
cvxpy==1.0.31
opencv-contrib-python==4.1.2.30
opencv-python==4.1.2.30


In [ ]:
!pip freeze | grep opencv

opencv-contrib-python==4.1.2.30
opencv-python==4.1.2.30


In [ ]:
!pip freeze | grep numpy

numpy==1.21.5


## Evaluation loop

In [ ]:
side_size = 256
mean = [0, 0, 0]
std = [1, 1, 1]
crop_size = 256
num_frames = 32
sampling_rate = 2
frames_per_second = 30
alpha = 4

class PackPathway(torch.nn.Module):
    """
    Transform for converting video frames as a list of tensors.
    """
    def __init__(self):
        super().__init__()

    def forward(self, frames: torch.Tensor):
        fast_pathway = frames
        # Perform temporal sampling from the fast pathway.
        slow_pathway = torch.index_select(
            frames,
            1,
            torch.linspace(
                0, frames.shape[1] - 1, frames.shape[1] // alpha
            ).long(),
        )
        frame_list = [slow_pathway, fast_pathway]
        return frame_list


def mp42tensor(video_path):
  '''
  Converts MP4 video to tensor format.
  :param 
      video_path: string, video file name
  :return 
      frames: numpy array
  '''
  videocap = cv2.VideoCapture(video_path)
  success, image = videocap.read()
  cnt = 0
  frames = []

  while success:
      image = resize(image, (224, 400))
      frames.append(image)
      success, image = videocap.read()
      cnt += 1
  frames = np.array(frames)
  return frames

# Transformation function applied to input videos
transform =  ApplyTransformToKey( # Applies transform to key of dictionary input
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(num_frames), # Uniformly subsamples indices from the temporal dimension of the video
            #Lambda(lambda x: x/255.0),
            NormalizeVideo(mean, std), # Normalization
            ShortSideScale(
                size=side_size
            ),
            #CenterCropVideo(crop_size),
            PackPathway()
        ]
    ),
)

In [ ]:
def evaluate(video_path, student, labels, transform=transform):
  '''
  Evaluation loop for calculating accuracy using test data
  :param video_path: string, video file name in test dataset
  :param student: Student/attacker model 
  :param labels: 
  :param transform: transformation to be applied on the video tensors
  '''
  vid_inp = torch.from_numpy(mp42tensor(video_path))
  # print(vid_inp.unsqueeze(0).shape)
  # pred_vic = api.movinet(vid_inp)
  # pred_vic = torch.tensor(pred_vic.numpy()).to(device)

  vid_inp = vid_inp.unsqueeze(0).to(device, dtype=torch.float32)
  # print(vid_inp.shape)

  pyt_in = vid_inp.permute(0, 4, 1, 2, 3).cpu()
  pred_stud = []

  for i in range(pyt_in.shape[0]):
    video_data = transform({"video": pyt_in[i]})
    inputs = video_data["video"]
    inputs = [inp.to(device)[None, ...] for inp in inputs]
    pred_stud.append(student(inputs))

  pred_stud = torch.cat(pred_stud, 0)
  pred_stud = pred_stud.to(device)
  # ind_vic = pred_vic.argmax(dim=1)
  ind_vic = pred_stud.argmax(dim=1)

  top5 = torch.topk(pred_stud.flatten(), 5).indices

  if labels in top5:
    return 1 
  return 0

In [ ]:
test = pd.read_csv('/content/testwithlabels.csv') # Loading the test dataset
print(len(test))
test.head()

In [ ]:
k600 = os.listdir('/content/drive/MyDrive/test') # Creating directory of the videos available in test dataset
print(len(k600))

As the length of k600 and test is different, we will find out the intersection of the video files from the both test and k600 which will be used for evaluation purposes

In [ ]:
id_match = []
count_ = 0

for i in range(len(k600)):
  if k600[i][:11] in test['youtube_id'].values: # Matching the first 11 characters of the video file name
    count_ += 1
    id_match.append(k600[i]) # Appending the names to a list
print(count_)

In [ ]:
new_id_match = []
for i in range(len(id_match)):
 new_id_match.append(id_match[i][:11]) # Appending the first 11 characters of the video file name

In [ ]:
dic = {}
for i in range(len(new_id_match)):
  dic[new_id_match[i]] = id_match[i] # Creating a dictionary of key value pairs denoting first 11 characters and full video names respectively

Now that we have the key value pairs for all the matching/intersecting video file names, let's create a new dataset with only those videos

In [ ]:
df = test[test['youtube_id'].isin(new_id_match)] # Creation of new dataframe for matching videos
df['id_new'] = df['youtube_id']
df.replace({'id_new':dic}, inplace=True) # remapping youtube ids with the complete video file name
df.to_csv('/content/df600.csv', index=False)

#### Loading the student model

In [ ]:
student_model = torch.load('/content/drive/MyDrive/interiit1503/movinet_weights/black_box_model.pth') ## Change according to path on machine

#### Running the evaluation loop

In [ ]:
root = '/content/drive/MyDrive/test/'
correct_pred = 0

for index, (id, label) in enumerate(tqdm(zip(df.id_new, df.id) , total=len(df))): # Iterating through the test dataframe
  correct_pred += evaluate(root + id, student_model, label)
  if index == 49:                
    break
print(f'Accuracy: {correct_pred/50}')